In [1]:
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

%matplotlib inline

import numpy as np
import pandas as pd
import time
import urllib.request
import sys
import os
from urllib.request import urlopen as url_open
import json #javascript object notation
from PIL import Image #Python Imaging Library
import cv2
from glob import glob

In [2]:
images_dir_path = 'images'
if not os.path.exists(images_dir_path):
    os.mkdir(images_dir_path)

In [3]:
from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [4]:
import contextlib
# https://api.nasa.gov/planetary/apod?date=2019-05-28&hd=True&api_key=TJbpneCHd96FEDuEItc91wrFkCfQUZVuY8pnRoCd
url = 'https://api.nasa.gov/planetary/apod?' # I don't get how url queries work, someone help
API_KEY = 'api_key=TJbpneCHd96FEDuEItc91wrFkCfQUZVuY8pnRoCd' #own API key, please register for your own at NASA Open APIs
HD = 'hd=True&'

start_date = date(2005, 5, 29)
end_date = date(2019, 5, 29)
date_list = []
imgs_url = []
count = 1
for single_date in daterange(start_date, end_date):
    DATE = 'date={}&'.format(single_date.strftime("%Y-%m-%d"))
    decoded_json = []
    with contextlib.closing(url_open(url +DATE+ HD + API_KEY)) as x: 
        decoded_string = x.read().decode('utf-8')
        decoded_json = json.loads(decoded_string)
    for key in decoded_json: #iterate through the json dictionar
        value = decoded_json[key]
    #         print('{}: {}'.format(key, value))
        if key == 'hdurl':
            imgs_url.append(value)
        elif key == 'date':
            date_list.append(value)
            sys.stdout.write('.')
            sys.stdout.flush()
            count += 1
            if count%100 == 0:
                sys.stdout.write('\n')
    time.sleep(2)
print('Done')

....................................

HTTPError: HTTP Error 500: INTERNAL SERVER ERROR

In [ ]:
def getData(url,name,date,file_path):
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:60.0) Gecko/20100101 Firefox/60.0'}
    page1=urllib.request.Request(url,headers=headers)
    data_file=urllib.request.urlopen(page1)
    print('Download [%s] %s '%(date,name), file_path)
    try:
        with open(file_path,'wb') as output:
            output.write(data_file.read())
            output.close()
    except Exception as e:
        print('Download %s Fail %s [%d]'%(name,str(e),count), file_path)
        return False
    return True

def get_all_data():
    count = 0
    image_list = []
    for d, img_url in zip(date_list,imgs_url):
        try:            
            basename = os.path.splitext(os.path.basename(img_url))
            new_name = '{}_{}{}'.format(d,basename[0],basename[1])
            image_file_path = os.path.join(images_dir_path, new_name)
            if False == getData(img_url,new_name,d,image_file_path):
                continue
            image_list.append(image_file_path)
        except Exception as e:
            print(str(e))
            time.sleep(10)
            continue
        time.sleep(1.1)
        
    # Save new_song list
    df_image = pd.DataFrame()
    df_image['date'] = d
    df_image['path'] = image_list
    df_image.to_csv(os.path.join('', 'image_list.csv'), index=False)
    print('Done')
    
get_all_data()